In [82]:
import sys

import matplotlib.pyplot as plt
import pandas as pd


sys.path.append("..")
from src.make_data import add_metrics, get_long_text_summary
# from src.plots import 

sys.tracebacklimit = 0 # easier to see problems
plt.rcParams["figure.figsize"] = (12, 8)


In [83]:
df = pd.read_csv("..//data/raw/the_zebra_movie_data.csv")
print(df.shape)


df = add_metrics(df)

(5043, 26)
(4998, 26)
shape - (4998, 30), columns added - ['lift', 'budget_log', 'gross_log', 'decade']


## Exploring Genre 

In [84]:
df.genres.head()

0    Action|Adventure|Fantasy|Sci-Fi
1           Action|Adventure|Fantasy
2          Action|Adventure|Thriller
3                    Action|Thriller
4                        Documentary
Name: genres, dtype: object

In [85]:
get_long_text_summary(df, "genres").sort_values("n", ascending=False).head(10)

IndexError: positional indexers are out-of-bounds

In [57]:
# also have the same format for plot k-w
df.plot_keywords.head()

0               avatar|future|marine|native|paraplegic
1    goddess|marriage ceremony|marriage proposal|pi...
2                  bomb|espionage|sequel|spy|terrorist
3    deception|imprisonment|lawlessness|police offi...
4                                                  NaN
Name: plot_keywords, dtype: object

In [58]:
get_long_text_summary(df, "plot_keywords")

,n,num_voted,movie_score,movie_score_median
kw_clean,,,,
wrongful imprisonment,1,1689764.0,9.3,9.3
prison cell search,1,1689764.0,9.3,9.3
rise to power,1,1155770.0,9.2,9.2
moral challenge,2,12590.5,9.1,9.1
meaning of life,2,12590.5,9.1,9.1
...,...,...,...,...
science experiment,1,25371.0,1.9,1.9
cnn reporter,1,534.0,1.9,1.9
pun,1,4377.0,1.7,1.7


In [59]:
df_explode = df.iloc[df["genres"].str.split("|").explode().index,:]
df_explode = df_explode.assign(genre_clean = df["genres"].str.split("|").explode())


len(df_explode["genre_clean"].value_counts()) # 26 unique categories

26

In [60]:
tmp = df_explode.groupby("genre_clean").size().reset_index(name="count").sort_values("count")

enough_genre = tmp[tmp["count"] > 50]['genre_clean'].to_list()

In [61]:
df_explode = df_explode.loc[df_explode["genre_clean"].isin(enough_genre), ]

In [67]:
df_explode.loc[df_explode["decade"] < 1960, "decade"] = 1960

grp_lift = df_explode.pivot_table(index="genre_clean", columns="decade", values = ["lift"], aggfunc=["median"]).fillna(0)
grp_lift.sort_values(('median', 'lift', 2010), ascending=False)


median                                                     
                  lift                                                     
decade            1960        1970       1980      1990      2000      2010
genre_clean                                                                
Music        21.088872   10.000000   3.674252  1.103642  1.038813  1.423998
Animation    33.508772    0.000000   1.923913  1.340204  1.080377  1.236392
Comedy        4.925532   23.533241   3.192717  1.206642  1.050477  1.216650
Horror       31.732582   95.333333   3.689270  1.415514  1.171595  1.197778
Family       13.025000    5.555556  11.350000  1.319169  1.333440  1.193939
Romance       6.257090    3.435294   2.478571  1.323942  0.862830  1.188298
Mystery       4.726281    2.351429   3.137714  1.202670  0.947855  1.172135
Biography     1.989359    4.562500   2.838172  0.917443  0.624663  1.042080
Adventure     4.925532    3.060317   1.878372  1.143658  0.965672  1.012069
Sci-Fi        4.726281    3.342857   2.215660  0.870028  0.833524  1.012069
Fantasy      32.423077    2.947948   3.192717  1.294758  0.982147  0.964256
Western       3.816667   45.961538   1.858462  1.070655  0.323201  0.947593
Drama         5.717647    4.562500   2.533333  1.096665  0.791771  0.942102
Crime         7.275000   18.048300   2.007407  1.133659  0.816140  0.934441
Thriller      7.275000   18.221667   2.224405  1.044853  0.896036  0.917430
Sport         0.000000  122.120049   6.633333  1.353340  0.770075  0.871783
Action        7.066667    2.609053   2.007407  1.038869  0.892209  0.842461
Musical       6.257090    3.048611   1.991429  1.083566  0.696902  0.830954
War           9.611606    0.580000   3.418533  0.620588  0.377107  0.784177
History       1.856018    1.266923   1.490556  0.541450  0.432084  0.504112
Documentary   0.000000   22.166667  41.914800  4.095140  1.225277  0.454936

In [78]:
df_explode.loc[df_explode["decade"] < 1960, "decade"] = 1960

grp_score = df_explode.pivot_table(index="genre_clean", columns="decade", values = ["movie_score"], aggfunc=["median"]).fillna(0)

grp_score.sort_values(('median', 'movie_score', 2010), ascending=False)

median                                   
            movie_score                                   
decade             1960  1970  1980  1990  2000 2010  9990
genre_clean                                               
Documentary        0.00  8.15  7.50  7.60  7.50  7.2  6.85
War                7.85  7.35  7.30  7.10  7.10  7.1  7.85
Sport              8.00  7.30  7.20  6.80  6.70  7.1  8.70
Biography          7.90  7.60  7.50  7.30  7.15  7.1  8.15
History            7.00  7.45  7.70  6.80  7.20  7.0  8.20
Animation          7.70  0.00  7.30  7.10  6.60  6.7  7.80
Drama              7.70  7.50  7.20  6.80  6.80  6.6  7.70
Adventure          7.20  6.80  6.70  6.40  6.30  6.6  7.75
Crime              7.65  7.50  6.60  6.60  6.50  6.5  7.90
Music              7.80  7.20  6.60  6.75  6.50  6.5  7.80
Sci-Fi             7.15  6.70  6.75  6.10  6.20  6.4  7.70
Western            7.40  7.50  7.10  6.70  6.50  6.4  0.00
Family             7.60  7.50  7.30  6.15  6.10  6.4  7.40
Mystery            7.85  7.60  6.70  6.70  6.50  6.4  7.50
Romance            7.40  7.30  6.60  6.65  6.50  6.4  7.30
Comedy             7.30  7.20  6.90  6.45  6.10  6.3  7.60
Action             7.40  6.70  6.60  6.20  6.20  6.3  7.40
Thriller           7.45  7.10  6.40  6.40  6.40  6.3  7.55
Fantasy            7.60  7.60  6.75  6.40  6.10  6.3  7.50
Musical            7.50  7.60  6.60  7.05  6.30  6.2  0.00
Horror             8.00  7.50  6.50  6.10  5.80  5.6  7.50

## Calc Growth Rates

In [72]:
grp_lift = grp_lift.assign(
    growth_since_1970 = (grp_lift[('median', 'lift', 2010)] - grp_lift[('median', 'lift', 1970)]) /grp_lift[('median', 'lift', 2010)] * 100,
    growth_since_1980 = (grp_lift[('median', 'lift', 2010)] - grp_lift[('median', 'lift', 1990)]) /grp_lift[('median', 'lift', 2010)] * 100,
    growth_since_1990 = (grp_lift[('median', 'lift', 2010)] - grp_lift[('median', 'lift', 1990)]) /grp_lift[('median', 'lift', 2010)] * 100,
    growth_since_2000 = (grp_lift[('median', 'lift', 2010)] - grp_lift[('median', 'lift', 2000)]) /grp_lift[('median', 'lift', 2010)] * 100
)

grp_lift.sort_values("growth_since_2000", ascending=False)

median                                                       \
                  lift                                                        
decade            1960        1970       1980      1990      2000      2010   
genre_clean                                                                   
Western       3.816667   45.961538   1.858462  1.070655  0.323201  0.947593   
War           9.611606    0.580000   3.418533  0.620588  0.377107  0.784177   
Biography     1.989359    4.562500   2.838172  0.917443  0.624663  1.042080   
Romance       6.257090    3.435294   2.478571  1.323942  0.862830  1.188298   
Music        21.088872   10.000000   3.674252  1.103642  1.038813  1.423998   
Mystery       4.726281    2.351429   3.137714  1.202670  0.947855  1.172135   
Sci-Fi        4.726281    3.342857   2.215660  0.870028  0.833524  1.012069   
Musical       6.257090    3.048611   1.991429  1.083566  0.696902  0.830954   
Drama         5.717647    4.562500   2.533333  1.096665  0.791771  0.942102   
History       1.856018    1.266923   1.490556  0.541450  0.432084  0.504112   
Comedy        4.925532   23.533241   3.192717  1.206642  1.050477  1.216650   
Crime         7.275000   18.048300   2.007407  1.133659  0.816140  0.934441   
Animation    33.508772    0.000000   1.923913  1.340204  1.080377  1.236392   
Sport         0.000000  122.120049   6.633333  1.353340  0.770075  0.871783   
Adventure     4.925532    3.060317   1.878372  1.143658  0.965672  1.012069   
Thriller      7.275000   18.221667   2.224405  1.044853  0.896036  0.917430   
Horror       31.732582   95.333333   3.689270  1.415514  1.171595  1.197778   
Fantasy      32.423077    2.947948   3.192717  1.294758  0.982147  0.964256   
Action        7.066667    2.609053   2.007407  1.038869  0.892209  0.842461   
Family       13.025000    5.555556  11.350000  1.319169  1.333440  1.193939   
Documentary   0.000000   22.166667  41.914800  4.095140  1.225277  0.454936   

            growth_since_1970 growth_since_1980 growth_since_1990  \
                                                                    
decade                                                              
genre_clean                                                         
Western          -4750.347081        -12.986853        -12.986853   
War                 26.037073         20.861176         20.861176   
Biography         -337.826401         11.960394         11.960394   
Romance           -189.093594        -11.414976        -11.414976   
Music             -602.248201         22.496958         22.496958   
Mystery           -100.610728         -2.605046         -2.605046   
Sci-Fi            -230.299222         14.034777         14.034777   
Musical           -266.880846        -30.400275        -30.400275   
Drama             -384.289541        -16.406251        -16.406251   
History           -151.317537         -7.406610         -7.406610   
Comedy           -1834.264873          0.822647          0.822647   
Crime            -1831.454996        -21.319508        -21.319508   
Animation          100.000000         -8.396446         -8.396446   
Sport           -13908.078635        -55.238179        -55.238179   
Adventure         -202.382194        -13.001968        -13.001968   
Thriller         -1886.164050        -13.889153        -13.889153   
Horror           -7859.179982        -18.178301        -18.178301   
Fantasy           -205.722660        -34.275449        -34.275449   
Action            -209.694001        -23.313483        -23.313483   
Family            -365.313267        -10.488861        -10.488861   
Documentary      -4772.482017       -800.157715       -800.157715   

            growth_since_2000  
                               
decade                         
genre_clean                    
Western             65.892469  
War                 51.910404  
Biography           40.056123  
Romance             27.389420  
Music               27.049514  
Mystery             19.134

In [79]:
grp_score = grp_lift.assign(
    growth_since_1970 = (grp_score[('median', 'movie_score', 2010)] - grp_score[('median', 'movie_score', 1970)]) /grp_score[('median', 'movie_score', 2010)] * 100,
    growth_since_1980 = (grp_score[('median', 'movie_score', 2010)] - grp_score[('median', 'movie_score', 1990)]) /grp_score[('median', 'movie_score', 2010)] * 100,
    growth_since_1990 = (grp_score[('median', 'movie_score', 2010)] - grp_score[('median', 'movie_score', 1990)]) /grp_score[('median', 'movie_score', 2010)] * 100,
    growth_since_2000 = (grp_score[('median', 'movie_score', 2010)] - grp_score[('median', 'movie_score', 2000)]) /grp_score[('median', 'movie_score', 2010)] * 100
)

grp_score.sort_values("growth_since_2000", ascending=False)

median                                                       \
                  lift                                                        
decade            1960        1970       1980      1990      2000      2010   
genre_clean                                                                   
Sport         0.000000  122.120049   6.633333  1.353340  0.770075  0.871783   
Family       13.025000    5.555556  11.350000  1.319169  1.333440  1.193939   
Adventure     4.925532    3.060317   1.878372  1.143658  0.965672  1.012069   
Comedy        4.925532   23.533241   3.192717  1.206642  1.050477  1.216650   
Fantasy      32.423077    2.947948   3.192717  1.294758  0.982147  0.964256   
Sci-Fi        4.726281    3.342857   2.215660  0.870028  0.833524  1.012069   
Action        7.066667    2.609053   2.007407  1.038869  0.892209  0.842461   
Animation    33.508772    0.000000   1.923913  1.340204  1.080377  1.236392   
Music        21.088872   10.000000   3.674252  1.103642  1.038813  1.423998   
Crime         7.275000   18.048300   2.007407  1.133659  0.816140  0.934441   
War           9.611606    0.580000   3.418533  0.620588  0.377107  0.784177   
Biography     1.989359    4.562500   2.838172  0.917443  0.624663  1.042080   
Mystery       4.726281    2.351429   3.137714  1.202670  0.947855  1.172135   
Western       3.816667   45.961538   1.858462  1.070655  0.323201  0.947593   
Romance       6.257090    3.435294   2.478571  1.323942  0.862830  1.188298   
Thriller      7.275000   18.221667   2.224405  1.044853  0.896036  0.917430   
Musical       6.257090    3.048611   1.991429  1.083566  0.696902  0.830954   
History       1.856018    1.266923   1.490556  0.541450  0.432084  0.504112   
Drama         5.717647    4.562500   2.533333  1.096665  0.791771  0.942102   
Horror       31.732582   95.333333   3.689270  1.415514  1.171595  1.197778   
Documentary   0.000000   22.166667  41.914800  4.095140  1.225277  0.454936   

            growth_since_1970 growth_since_1980 growth_since_1990  \
                                                                    
decade                                                              
genre_clean                                                         
Sport               -2.816901          4.225352          4.225352   
Family             -17.187500          3.906250          3.906250   
Adventure           -3.030303          3.030303          3.030303   
Comedy             -14.285714         -2.380952         -2.380952   
Fantasy            -20.634921         -1.587302         -1.587302   
Sci-Fi              -4.687500          4.687500          4.687500   
Action              -6.349206          1.587302          1.587302   
Animation          100.000000         -5.970149         -5.970149   
Music              -10.769231         -3.846154         -3.846154   
Crime              -15.384615         -1.538462         -1.538462   
War                 -3.521127          0.000000          0.000000   
Biography           -7.042254         -2.816901         -2.816901   
Mystery            -18.750000         -4.687500         -4.687500   
Western            -17.187500         -4.687500         -4.687500   
Romance            -14.062500         -3.906250         -3.906250   
Thriller           -12.698413         -1.587302         -1.587302   
Musical            -22.580645        -13.709677        -13.709677   
History             -6.428571          2.857143          2.857143   
Drama              -13.636364         -3.030303         -3.030303   
Horror             -33.928571         -8.928571         -8.928571   
Documentary        -13.194444         -5.555556         -5.555556   

            growth_since_2000  
                               
decade                         
genre_clean                    
Sport                5.633803  
Family               4.687500  
Adventure            4.545455  
Comedy               3.174603  
Fantasy              3.174603  
Sci-Fi               3.125